<a href="https://colab.research.google.com/github/Alebasso01/Pacman/blob/master/dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pandasql
!pip install -q dash jupyter-dash
from pandasql import sqldf
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from jupyter_dash import JupyterDash
from dash import Dash, html, dcc, Input, Output

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26771 sha256=c4d5093a98eb26025daaa016b7cd0dcf1252f500b851f34e1514ccf0df68c606
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.4 MB/s eta 0:00:00


In [ ]:
df = pd.read_csv("drive/MyDrive/superstore/Superstore.csv",encoding='latin1')

In [ ]:
df.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')

In [ ]:
df.drop_duplicates(inplace = True)
df = df.drop(["Row ID","Order ID","Customer ID","Customer Name",],axis=1)


In [ ]:
df = df.drop(["Country","Postal Code","Product ID"],axis = 1)

In [ ]:
df.columns = df.columns.str.replace('Ship Mode', 'ShipMode')
df.columns = df.columns.str.replace('Sub-Category', 'SubCategory')
df.columns = df.columns.str.replace('Order Date', 'OrderDate')
df.columns = df.columns.str.replace('Ship Date', 'ShipDate')
df.columns = df.columns.str.replace('Product Name', 'ProductName')
state_name_code = {'Alabama': 'AL','Alaska': 'AK','Arizona': 'AZ','Arkansas': 'AR','California': 'CA','Colorado': 'CO',
                   'Connecticut': 'CT','Delaware': 'DE','Florida': 'FL','Georgia': 'GA','Hawaii': 'HI','Idaho': 'ID',
                   'Illinois': 'IL','Indiana': 'IN','Iowa': 'IA','Kansas': 'KS','Kentucky': 'KY','Louisiana': 'LA',
                   'Maine': 'ME','Maryland': 'MD','Massachusetts': 'MA','Michigan': 'MI','Minnesota': 'MN',
                   'Mississippi': 'MS','Missouri': 'MO','Montana': 'MT','Nebraska': 'NE','Nevada': 'NV',
                   'New Hampshire': 'NH','New Jersey': 'NJ','New Mexico': 'NM','New York': 'NY',
                   'North Carolina': 'NC','North Dakota': 'ND','Ohio': 'OH','Oklahoma': 'OK',
                   'Oregon': 'OR','Pennsylvania': 'PA','Rhode Island': 'RI','South Carolina': 'SC',
                   'South Dakota': 'SD','Tennessee': 'TN','Texas': 'TX','Utah': 'UT','Vermont': 'VT',
                   'Virginia': 'VA','District of Columbia': 'WA','Washington': 'WA','West Virginia': 'WV',
                   'Wisconsin': 'WI','Wyoming': 'WY'}
df['state_name_code'] = df.State.apply(lambda x: state_name_code[x])
df


,OrderDate,ShipDate,ShipMode,Segment,City,State,Region,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit,state_name_code
0,11/8/2016,11/11/2016,Second Class,Consumer,Henderson,Kentucky,South,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,KY
1,11/8/2016,11/11/2016,Second Class,Consumer,Henderson,Kentucky,South,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,KY
2,6/12/2016,6/16/2016,Second Class,Corporate,Los Angeles,California,West,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,CA
3,10/11/2015,10/18/2015,Standard Class,Consumer,Fort Lauderdale,Florida,South,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,FL
4,10/11/2015,10/18/2015,Standard Class,Consumer,Fort Lauderdale,Florida,South,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,FL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,1/21/2014,1/23/2014,Second Class,Consumer,Miami,Florida,South,Furniture,Furnishings,Ultra Door Pull Handle,25.2480,3,0.20,4.1028,FL
9990,2/26/2017,3/3/2017,Standard Class,Consumer,Costa Mesa,California,West,Furniture,Furnishings,Tenex B1-RE Series Chair Mats for Low Pile Car...,91.9600,2,0.00,15.6332,CA
9991,2/26/2017,3/3/2017,Standard Class,Consumer,Costa Mesa,California,West,Technology,Phones,Aastra 57i VoIP phone,258.5760,2,0.20,19.3932,CA
9992,2/26/2017,3/3/2017,Standard Class,Consumer,Costa Mesa,California,West,Office Supplies,Paper,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",29.6000,4,0.00,13.3200,CA


In [ ]:
df['date'] = pd.to_datetime(df['OrderDate'])
df['year']=df['date'].dt.year
df = df.drop(["date",],axis=1)
query = "SELECT  City,Region,Category,SubCategory,Profit,Sales,Quantity,SUM(Sales),SUM(Profit) FROM df GROUP BY City,Category,SubCategory ORDER BY SUM(Sales) DESC"
df_City = sqldf(query)
query2= "SELECT * FROM df_City LIMIT 0, 10"
df_City = sqldf(query2)
df20 = df.loc[(df.City == "Los Angeles")|(df.City == "New York City")|(df.City == "Seattle")|(df.City == "San Francisco")|(df.City == "Philadelphia")|(df.City == "Houston")|(df.City == "Chicago")|(df.City == "San Diego")|(df.City == "Jacksonville")|(df.City == "Detroit")]
query = "SELECT City,Region,Category,SubCategory,SUM(Sales),SUM(Profit),SUM(Quantity) FROM df20 GROUP BY City,Category,SubCategory ORDER BY SUM(Profit) DESC"
df21 = sqldf(query)
df.drop_duplicates(inplace = True)
df2 = df.drop(["Segment", "City", "State", "Region", "Category", "SubCategory","state_name_code","Discount"],axis=1)
query = "SELECT  City,Profit,Sales,Quantity,SUM(Sales),SUM(Profit),Category FROM df GROUP BY City ORDER BY SUM(Sales) DESC"
df6 = sqldf(query)
query2= "SELECT * FROM df6 LIMIT 0, 10"
df6 = sqldf(query2)
df11 = df6.drop(["City","Quantity","Category","SUM(Sales)","SUM(Profit)"],axis = 1)
state_data = df[['state_name_code','Sales','Profit']].groupby(['state_name_code']).sum()
df2 = df2.drop(["ShipMode","OrderDate"],axis=1)
df3 = df.sort_values('Category', ascending=False)
df2= df2.drop(["ShipDate","ProductName","year"],axis = 1)

SyntaxError: ignored

In [ ]:
df['date'] = pd.to_datetime(df['OrderDate'])
df['year']=df['date'].dt.year
df = df.drop(["date",],axis=1)

In [ ]:
df1 = df.query("SELECT SUM(Profit),SUM(Quantity),SUM(Sales),year,Category,SubCategory FROM df GROUP BY year,Category ORDER BY year ")

ObjectNotExecutableError: ignored

In [ ]:
df.value_counts("ShipMode")

ShipMode
Standard Class    5968
Second Class      1945
First Class       1538
Same Day           543
dtype: int64

In [ ]:
app = JupyterDash(__name__)
app.layout = html.Div([
    dcc.Markdown(children='---'),
    dcc.Markdown(children='*profitti sottocategorie*'),
    dcc.Markdown(children='---'),
    dcc.Markdown(children='quale citta?'),
    dcc.Dropdown(list(df2.columns), value = 'Sales', id = 'input_value',),
    dcc.Markdown(children='---'),
    dcc.Markdown(children='---'),
    dcc.Graph(id = 'graph_line'),
    dcc.Markdown(children='---'),
])
@app.callback(
    Output(component_id = 'graph_line', component_property = 'figure'),
    Input(component_id = 'input_value', component_property = 'value'),
)
def update_input_text(input_value):
  if (input_value == "Profit"):
    val_y = "SUM(Profit)"
  elif (input_value == "Sales"):
    val_y = "SUM(Sales)"
  elif(input_value == "Quantity"):
    val_y = "SUM(Quantity)"
  fig = px.line(data_frame = df1,
                x = "year",
                y = val_y,
                color = "Category",
                markers=True,
                )
  return fig
app.run_server(mode = 'inline')

NameError: ignored

In [ ]:
state_data = df[['Profit', 'state_name_code','Sales']].groupby(['state_name_code']).sum()
app = JupyterDash(__name__)
app.layout = html.Div(
                     children=[
                      html.Div(
                          children= [
                              html.H1("SUPERSTORES IN USA"),
                            ],style={'text-align': "center", 'color': "white", 'font-family': "Arial",'background-color': "#5f819e"}
                      ),
#-------------------------------------------------------1e2-------------------------------------------------------------------
                         html.Div(
                             children=[
#-------------------------------------------------------BOX 1-------------------------------------------------------------------------------
                         html.Div(
                            children=[
                            html.Div(
                              children=[
                              html.Div(
                                  children=[
                                    html.P('Choose between:'),

                                    ], style= {'font-size': 20,'font-family': "Arial", 'left':0, 'width':180, 'color': "#284B63"}
                                    ),
                                    html.Div(
                                  children=[
                                      dcc.Dropdown(list(df11.columns), value = 'Sales', id = 'input_value',)
                                    ], style= {'font-size': 20,'color': "#284B63",'position':"relative",'top':15, 'left':0, 'width': 370}
                                    ),],style= {'display': "flex"}),
                              dcc.Graph(id = 'graph1'),
                            ],style= {'width': 550}
                         ),
#---------------------------------------------------------BOX 2---------------------------------------------------------------------------
                         html.Div(
                            children=[
                            html.Div(
                              children=[
                              html.Div(
                                  children=[
                                    html.P('Choose between:'),

                                    ], style= {'font-size': 20,'font-family': "Arial", 'left':0, 'width':180, 'color': "#284B63"}
                                    ),
                                    html.Div(
                                  children=[
                                    dcc.Dropdown(list(df2.columns), value = 'Profit', id = 'input_spq',),
                                    ], style= {'font-size': 20,'color': "#284B63",'position':"relative",'top':15, 'left':0, 'width': 370}
                                    ),],style= {'display': "flex"}),
                              dcc.Graph(id = 'graph2'),
                            ],style= { 'width': 550 }
                         ),
                         ],style= {'display': "inline-block"}),
#--------------------------------------------------------------3 e 4-------------------------------------------------------------------------------
                             html.Div(
                             children=[
#--------------------------------------------------------------BOX 3-------------------------------------------------------------------------------
                            html.Div(
                            children=[
                            html.Div(
                              children=[
                              html.Div(
                                  children=[
                                    html.P('Choose the region:'),

                                    ], style= {'font-size': 20,'font-family': "Arial", 'left':0, 'width':200, 'color': "#284B63"}
                                    ),
                                    html.Div(
                                  children=[
                                    dcc.Dropdown(list(df.Region.unique()), value = 'East', id = 'input_region'),
                                    ], style= {'font-size': 20,'color': "#284B63",'position':"relative",'top':15, 'left':0, 'width': 400}
                                    ),],style= {'display': "flex"}),
                              dcc.Graph(id = 'graph3'),
                            ],style= {'position': "absolute",'width': 550, 'top': 78, 'left':592}
                         ),
#--------------------------------------------------------------BOX 4-------------------------------------------------------------------------------
                             html.Div(
                            children=[
                            html.Div(
                              children=[
                              html.Div(
                                  children=[
                                    html.P('Choose the city:'),

                                    ], style= {'font-size': 20,'font-family': "Arial", 'left':0, 'width':160, 'color': "#284B63" }
                                    ),
                                    html.Div(
                                  children=[
                                    dcc.Dropdown(list(df21.City.unique()), value = 'New York City', id = 'input_City')
                                    ], style= {'font-size': 20,'color': "#284B63",'position':"relative",'top':15, 'left':0, 'width': 390}
                                    ),],style= {'display': "flex"}),
                              dcc.Graph(id = 'graph4'),
                            ],style= {'width': 550,'position': "absolute", 'top': 347, 'left':592}
                         ),
                         ],style= {'display': "inline-block"}),
], style= {'height': 615, 'width': 1143, 'position': "absolute", 'top': 0, 'left': 0, 'rigth': 0, 'background-color' :"#f3f5f6",
})#-------------DIV PRINCIPALE-----------------------------------------------------------
#GRAFICO 1-----------------------------------------------------------------------------------------------------------------------------------------
@app.callback(
    Output(component_id = 'graph1', component_property = 'figure'),
    Input(component_id = 'input_value', component_property = 'value')
    )
def update_figure_output(input_value):
  fig1 = go.Figure(data=go.Choropleth(
                  locations = state_data.index,
                  z = state_data[input_value],
                  locationmode = 'USA-states',
                  colorscale = 'blues',
                  ))

  fig1.update_layout(
    #title_text = 'Profit/Sales for each state',
    geo_scope='usa',
    height=205,
    margin=dict(l=0, r=0, t=0, b=0),
    paper_bgcolor="#f3f5f6",
  )

  return fig1

#GRAFICO 2-----------------------------------------------------------------------------------------------------------------------------------------
@app.callback(
    Output(component_id = 'graph2', component_property = 'figure'),
    Input(component_id = 'input_spq', component_property = 'value'),
)
def update_input_text(input_spq):
  if (input_spq == "Profit"):
    val_y = "SUM(Profit)"
  elif (input_spq == "Sales"):
    val_y = "SUM(Sales)"
  elif(input_spq == "Quantity"):
    val_y = "SUM(Quantity)"
  fig2 = px.line(data_frame = df1,
                x = "year",
                y = val_y,
                width= 550,
                color = "Category",
                color_discrete_map = {'Technology':'#905a51','Office Supplies':'#e76941','Furniture':'#ffb315'},
                markers=True)

  fig2.update_traces(line=dict(width=7))


  fig2.update_layout(
    height=205,
    margin=dict(l=0, r=0, t=0, b=0),
    paper_bgcolor="#f3f5f6",
    plot_bgcolor="#f3f5f6",
  )
  return fig2
#GRAFICO 3-----------------------------------------------------------------------------------------------------------------------------------------
@app.callback(
    Output(component_id = 'graph3', component_property = 'figure'),
    Input(component_id = 'input_region', component_property = 'value')
)

def update_input_text(input_region):
  fig3 = px.pie(data_frame = df[df.Region == input_region],
               values = "Quantity",
               names = "ShipMode",
               #title = "Torta",
               color = "ShipMode",
               color_discrete_map = {'First Class':'#d4e9c6','Second Class':'#cfb072','Standard Class':'#99a968','Same Day':'#4d8d53'},
               width=550,
               hole= 0.5

  )
  fig3.update_layout(
    height=205,
    paper_bgcolor="#f3f5f6",
    margin=dict(l=0, r=0, t=0, b=0),
    annotations=[dict(text=input_region, x=0.5, y=0.5, font_size=20, showarrow=False),],
  )
  return fig3
#GRAFICO 4-----------------------------------------------------------------------------------------------------------------------------------------
@app.callback(
    Output(component_id = 'graph4', component_property = 'figure'),
    Input(component_id = 'input_City', component_property = 'value'),
)

def update_input_text(input_City):
  fig4 = px.bar(data_frame = df21[df21.City == input_City],
               x = "SubCategory",
               y = "SUM(Profit)",
               color = "Category",
               width=550,
               color_discrete_map = {'Technology':'#905a51','Office Supplies':'#e76941','Furniture':'#ffb315'},
  )
  fig4.update_layout(
    height=205,
    margin=dict(l=0, r=0, t=0, b=0),
    paper_bgcolor="#f3f5f6",
    plot_bgcolor="#f3f5f6",
  )
  return fig4




#RUN----------------------------------------------------------------------------------------------------------------------------------------------
if __name__ == "__main__":
    app.run_server(debug=True)

In [ ]:
state_data = df[['Sales', 'state_name_code']].groupby(['state_name_code']).sum()
fig = go.Figure(data=go.Choropleth(
     locations=state_data.index,
     z = state_data.Sales,
     locationmode =  'USA-states',
      colorscale = 'Blues',
       colorbar_title = 'Sales ',
))
fig.update_layout(
    title_text = 'Deliveries in USA',
        geo_scope='usa',
        height = 400
)



fig.show()

In [ ]:
fig3 = px.pie(data_frame = df,
               values = "Quantity",
               names = "ShipMode",
               title = "Torta",
               color = "ShipMode",
               color_discrete_map = {'First Class':'#d4e9c6','Second Class':'#cfb072','Standard Class':'#99a968','Same Day':'#4d8d53'},
               width=850,
               hole= 0.2,


  )
fig3.update_layout(
    height=305,
    margin=dict(l=0, r=0, t=0, b=0),
  )
fig3.show()

In [ ]:
df1

In [ ]:
fig2 = px.line(data_frame = df1,
        x = df1.year,
        y = "SUM(Profit)",
        color = "Category",
        color_discrete_map = {'Technology':'#905a51','Office Supplies':'#e76941','Furniture':'#ffb315'},
        markers=True,
        width = 650)
fig2.update_traces(line=dict(width=7))
fig2.update_layout(
    height=505,
    margin=dict(l=0, r=0, t=0, b=0),
    paper_bgcolor="#f3f5f6",
    plot_bgcolor="#f3f5f6",
  )

fig2.show()

In [ ]:
 fig4 = px.bar(data_frame = df21[df21.City == "Los Angeles"],
               x = "SubCategory",
               y = "SUM(Profit)",
               color = "Category",
               width=1250,
               title = "LOS ANGELE ",
               color_discrete_map = {'Technology':'#905a51','Office Supplies':'#e76941','Furniture':'#ffb315'},
  )
fig4.update_layout(
    height=505,
    margin=dict(l=0, r=0, t=110, b=0),
    paper_bgcolor="#f3f5f6",
    plot_bgcolor="#f3f5f6",

  )
fig4.show()